In [ ]:
import ollama
from ollama import chat
from pydantic import BaseModel
import re
import pandas as pd
import torch
import sys
import os
import json

from neurovlm.data import get_data_dir
from neurovlm.models import Specter

## Load data and models

In [3]:
# Load data and Specter
data_dir = get_data_dir()
# Try reading with pyarrow engine and fallback to fastparquet if error occurs
try:
	df = pd.read_parquet(data_dir / "publications.parquet", engine="pyarrow")
except Exception as e:
	print(f"pyarrow failed: {e}, trying fastparquet...")
	df = pd.read_parquet(data_dir / "publications.parquet", engine="fastparquet")
specter = Specter()
aligner = torch.load(data_dir / "aligner.pt", weights_only=False).to("cpu")
latent_text = torch.load(data_dir / "latent_text.pt", weights_only=True).to("cpu")
latent_text = latent_text / latent_text.norm(dim=1, keepdim=True)  # unit norm

There are adapters available but none are activated for the forward pass.


## Example query

The titles and abstract most related to the query will be passed to the LM.

In [4]:
# Encode query with specter than rank publications
query = "what is the role of the hippocampus in memory formation"
encoded_text = specter(query)[0].detach()
encoded_text_norm = encoded_text / encoded_text.norm()
cos_sim = latent_text @ encoded_text_norm
inds = torch.argsort(cos_sim, descending=True)

# Aggregate publications to pass to LM
papers = "\n".join(
    [f"[{ind + 1}] " + df.iloc[int(i)]["name"] + "\n" + re.sub(r'\s+', ' ', df.iloc[int(i)]["description"].replace("\n", "")) + "\n"
     for ind, i in enumerate(inds[:5])]
)

In [5]:
# Top 10 related publications - these will be passed to LM
df.iloc[inds[:10]]["name"].values.tolist()

['Factors affecting the hippocampal BOLD response during spatial memory.',
 'Role of hippocampal CA1 atrophy in memory encoding deficits in amnestic Mild Cognitive Impairment.',
 'Memory related dysregulation of hippocampal function in major depressive disorder.',
 'Hippocampal functional connectivity and episodic memory in early childhood.',
 'Hippocampal activation for autobiographical memories over the entire lifetime in  healthy aged subjects: an fMRI study.',
 'Long-term retrograde amnesia...the crucial role of the hippocampus.',
 'Memory in frontal lobe epilepsy: an fMRI study.',
 'The hippocampus remains activated over the long term for the retrieval of truly episodic memories.',
 'The stressed hippocampus, synaptic plasticity and lost memories.',
 'Probing the relevance of the hippocampus for conflict-induced memory improvement']

In [49]:
system_prompt = """
You are a helpful neuroscience research assistant.  
You will receive a set of publications and a user query. Your task is to summarize key findings and insights from these publications, focusing on how they relate to the query.

Your response must:
- **Start with a brief overview** (2-4 sentences) summarizing the main themes or takeaways across the publications .  
- Be **entirely based on the information in the publications** and how it **directly ties to the user's query**. Do not add outside knowledge or speculation.  
- **Identify how each publication relates to the query**. If the publications **directly answer the query**, state the answer clearly. If they **do not answer it fully**, highlight **relevant points, evidence, or gaps** that inform the query.  
- **Synthesize across studies**, noting:  
  - Key areas of agreement or convergence.  
  - Conflicting or divergent findings, with a balanced summary and any contextual factors that may explain differences (e.g., methods, populations, analyses).  
- Use **paragraphs or bullet points** depending on the query:  
  - Bullet points → lists of findings, comparisons, or key points.  
  - Paragraphs → integrative or narrative summaries.  
- Maintain an **objective, precise, scholarly tone** suitable for neuroscience research contexts.  
"""

In [50]:
user_prompt = f"""
Here are some publications related to the query "{query}":
{papers}
"""

In [51]:
response = chat(
  messages=[
      {
        'role': 'system',
        'content': system_prompt
      },
      {
        'role': 'user',
        'content': user_prompt,
    }
  ],
  model='qwen2.5:3b-instruct' #'llama3.2:3b'
)

In [52]:
output_text = response['message']['content']

In [53]:
print(output_text)

### Overview

The provided publications cover various aspects of the hippocampal role in memory formation, including its function during spatial memory tasks, its relation to specific brain areas involved in spatial learning and navigation, its potential role in encoding versus retrieval deficits associated with mild cognitive impairment (MCI), its involvement in memory-related dysregulation in depression, its functional connectivity changes across childhood development, and its activation patterns for autobiographical memories over the lifespan. These studies provide a comprehensive view of how different research methodologies illuminate various facets of hippocampal function.

### Relation to Query

#### What is the Role of the Hippocampus in Memory Formation?

- **Spatial Memory:**
  - Publication [1] discusses the role of the hippocampus during spatial memory tasks, suggesting that while there are additional areas involved (such as parahippocampal gyrus and precuneus), the hippocam

In [ ]:
# should i remove talking about Synthesis and Conflicting or Divergent Findings? It might make the LLM respond faster